In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/BioInf-Final/')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np
import time
import random

from source.dataset import DataSet, prepare_interaction_pairs

def concordance_index(y_true, y_pred):
    pairs = [(i, j) for i in range(len(y_true)) for j in range(i+1, len(y_true)) if y_true[i] != y_true[j]]
    concordant = sum((y_true[i] > y_true[j]) == (y_pred[i] > y_pred[j]) for i, j in pairs)
    return concordant / len(pairs)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            drugs, proteins, labels = batch['drug'].to(device), batch['target'].to(device), batch['affinity'].to(device)
            # drugs, proteins, labels = [b.to(device) for b in batch]
            outputs = model(drugs, proteins)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    mse = total_loss / len(loader)
    ci = concordance_index(all_labels, all_preds)
    return mse, ci

def unite_lists(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

from torch.utils.data import DataLoader

def create_reproducible_dataloader(dataset, batch_size, shuffle=True, seed=42):
    generator = torch.Generator()
    generator.manual_seed(seed)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=0,  # Set to 0 for full reproducibility
        generator=generator,
        worker_init_fn=seed_worker
    )

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def run_experiment(model_class, config, train_dataset, test_dataset, dataset, num_epochs=100):
    set_seeds(seed=42)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Set up data loaders
    train_loader = create_reproducible_dataloader(train_dataset, batch_size=config['batch_size'])
    test_loader = create_reproducible_dataloader(test_dataset, batch_size=config['batch_size'], shuffle=False)

    # Initialize model
    model = model_class(config).to(device)

    # Define loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Training loop
    best_test_mse = float('inf')
    best_test_ci = 0

    for epoch in tqdm(range(num_epochs), desc="Epochs"):
    # for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            drugs, proteins, labels = batch['drug'].to(device), batch['target'].to(device), batch['affinity'].to(device)
            optimizer.zero_grad()
            outputs = model(drugs, proteins)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        # Evaluation
        model.eval()
        test_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_loader:
                drugs, proteins, labels = batch['drug'].to(device), batch['target'].to(device), batch['affinity'].to(device)
                outputs = model(drugs, proteins)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        test_loss /= len(test_loader)
        test_ci = concordance_index(all_labels, all_preds)

        if test_loss < best_test_mse:
            best_test_mse = test_loss
            best_test_ci = test_ci
            torch.save(model.state_dict(), f'{model_class.__name__}_{dataset}_best_model.pt')

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
              f"Test MSE: {test_loss:.4f}, Test CI: {test_ci:.4f}")

    print(f"Best Test MSE: {best_test_mse:.4f}, Best Test CI: {best_test_ci:.4f}")
    return best_test_mse, best_test_ci

In [ ]:
class DTIDataset(Dataset):
    def __init__(self, drugs, targets, affinity):
        self.drugs = torch.LongTensor(drugs)
        self.targets = torch.LongTensor(targets)
        self.affinity = torch.FloatTensor(affinity)

    def __len__(self):
        return len(self.affinity)

    def __getitem__(self, idx):
        return {
            'drug': self.drugs[idx],
            'target': self.targets[idx],
            'affinity': self.affinity[idx]
        }

def get_datasets(dataset_type, config):
  if dataset_type == 'davis':
      dataset = DataSet(config['davis_path'], config['problem_type'], config['max_seq_len'],
                        config['max_smi_len'], 'davis', config['davis_convert_to_log'])
  else:
      dataset = DataSet(config['kiba_path'], config['problem_type'], config['max_seq_len'],
                        config['max_smi_len'], 'kiba', config['kiba_convert_to_log'])

  XD, XT, Y, label_row_inds, label_col_inds, test_fold, train_folds = dataset.get_data()

  train_fold = unite_lists(train_folds)

  train_drugs, train_targets, train_affinity = prepare_interaction_pairs(XD, XT, Y, label_row_inds[train_fold], label_col_inds[train_fold])
  test_drugs, test_targets, test_affinity = prepare_interaction_pairs(XD, XT, Y, label_row_inds[test_fold], label_col_inds[test_fold])

  train_dataset = DTIDataset(train_drugs, train_targets, train_affinity)
  test_dataset = DTIDataset(test_drugs, test_targets, test_affinity)
  return train_dataset, test_dataset

In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, config):
        super(SimpleLSTM, self).__init__()
        self.drug_embedding = nn.Embedding(config['charsmiset_size'] + 1, config['embed_dim'])
        self.protein_embedding = nn.Embedding(config['charseqset_size'] + 1, config['embed_dim'])

        # Single layer LSTM with reduced hidden size
        self.drug_lstm = nn.LSTM(config['embed_dim'], config['lstm_dim'], num_layers=config['lstm_layers'], batch_first=True)
        self.protein_lstm = nn.LSTM(config['embed_dim'], config['lstm_dim'], num_layers=config['lstm_layers'], batch_first=True)

        # Reduced number of fully connected layers
        self.fc1 = nn.Linear(config['lstm_dim'] * 2, 32)  # 64 because we concatenate two 32-dim vectors
        self.fc2 = nn.Linear(32, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(config['dropout_rate'])

    def forward(self, drug, protein):
        drug_embedded = self.drug_embedding(drug)
        protein_embedded = self.protein_embedding(protein)

        _, (drug_hidden, _) = self.drug_lstm(drug_embedded)
        _, (protein_hidden, _) = self.protein_lstm(protein_embedded)

        # Use only the last hidden state
        drug_hidden = drug_hidden[-1]
        protein_hidden = protein_hidden[-1]

        concat = torch.cat([drug_hidden, protein_hidden], dim=1)

        fc1 = self.relu(self.fc1(concat))
        fc1 = self.dropout(fc1)

        output = self.fc2(fc1).squeeze(-1)
        return output

config = {
  'charsmiset_size': 64,
  'charseqset_size': 26,
  'embed_dim': 128,
  'lstm_dim': 64,
  'lstm_layers': 2,
  'dropout_rate': 0.3,
  'learning_rate': 0.001,
  'batch_size': 128,

  # Dataset specific
  'davis_convert_to_log': True,
  'kiba_convert_to_log': False,

  # Input dimensions
  'max_seq_len': 1000,
  'max_smi_len': 100,
  'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
  'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
  'problem_type': 1

}

train_dataset, test_dataset = get_datasets('davis', config)

# Run experiment with SimpleLSTM
mse, ci = run_experiment(SimpleLSTM, config, train_dataset, test_dataset, num_epochs=100)
print(f"SimpleLSTM - Final MSE: {mse:.4f}, Final CI: {ci:.4f}")

Epoch 1/100, Train Loss: 3.2424, Test MSE: 0.7971, Test CI: 0.5373
Epoch 2/100, Train Loss: 1.4436, Test MSE: 0.7826, Test CI: 0.5644
Epoch 3/100, Train Loss: 1.4412, Test MSE: 0.7781, Test CI: 0.5782
Epoch 4/100, Train Loss: 1.4116, Test MSE: 0.8006, Test CI: 0.5729
Epoch 5/100, Train Loss: 1.4109, Test MSE: 0.7697, Test CI: 0.5857
Epoch 6/100, Train Loss: 1.4039, Test MSE: 0.7608, Test CI: 0.5931
Epoch 7/100, Train Loss: 1.3850, Test MSE: 0.7591, Test CI: 0.5878
Epoch 8/100, Train Loss: 1.3563, Test MSE: 0.7659, Test CI: 0.5821
Epoch 9/100, Train Loss: 1.3480, Test MSE: 0.7712, Test CI: 0.5911
Epoch 10/100, Train Loss: 1.3279, Test MSE: 0.7644, Test CI: 0.5901
Epoch 11/100, Train Loss: 1.3016, Test MSE: 0.7826, Test CI: 0.5932
Epoch 12/100, Train Loss: 1.2705, Test MSE: 0.7602, Test CI: 0.5986
Epoch 13/100, Train Loss: 1.2447, Test MSE: 0.8390, Test CI: 0.6018
Epoch 14/100, Train Loss: 1.2292, Test MSE: 0.7494, Test CI: 0.6000
Epoch 15/100, Train Loss: 1.2167, Test MSE: 0.7544, Test 

In [ ]:
class BidirectionalLSTM(nn.Module):
    def __init__(self, config):
        super(BidirectionalLSTM, self).__init__()
        self.drug_embedding = nn.Embedding(config['charsmiset_size'] + 1, config['embed_dim'])
        self.protein_embedding = nn.Embedding(config['charseqset_size'] + 1, config['embed_dim'])

        self.drug_lstm = nn.LSTM(config['embed_dim'], config['lstm_dim'],
                                 num_layers=config['lstm_layers'], batch_first=True, bidirectional=True)
        self.protein_lstm = nn.LSTM(config['embed_dim'], config['lstm_dim'],
                                    num_layers=config['lstm_layers'], batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(config['lstm_dim'] * 4, 64)
        self.fc2 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(config['dropout_rate'])

    def forward(self, drug, protein):
        drug_embedded = self.drug_embedding(drug)
        protein_embedded = self.protein_embedding(protein)

        _, (drug_hidden, _) = self.drug_lstm(drug_embedded)
        _, (protein_hidden, _) = self.protein_lstm(protein_embedded)

        drug_hidden = torch.cat((drug_hidden[-2], drug_hidden[-1]), dim=1)
        protein_hidden = torch.cat((protein_hidden[-2], protein_hidden[-1]), dim=1)

        concat = torch.cat([drug_hidden, protein_hidden], dim=1)

        fc1 = self.relu(self.fc1(concat))
        fc1 = self.dropout(fc1)

        output = self.fc2(fc1).squeeze(-1)
        return output

config = {
  'charsmiset_size': 64,
  'charseqset_size': 26,
  'embed_dim': 128,
  'lstm_dim': 64,
  'lstm_layers': 2,
  'dropout_rate': 0.3,
  'learning_rate': 0.0005,
  'batch_size': 128,

  # Dataset specific
  'davis_convert_to_log': True,
  'kiba_convert_to_log': False,

  # Input dimensions
  'max_seq_len': 1000,
  'max_smi_len': 100,
  'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
  'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
  'problem_type': 1

}

train_dataset, test_dataset = get_datasets('davis', config)

# Run experiment with BidirectionalLSTM
mse, ci = run_experiment(BidirectionalLSTM, config, train_dataset, test_dataset, num_epochs=100)
print(f"BidirectionalLSTM - Final MSE: {mse:.4f}, Final CI: {ci:.4f}")

Epoch 1/100, Train Loss: 3.8459, Test MSE: 0.7826, Test CI: 0.5918
Epoch 2/100, Train Loss: 1.1685, Test MSE: 0.6755, Test CI: 0.7141
Epoch 3/100, Train Loss: 1.0384, Test MSE: 0.6323, Test CI: 0.7431
Epoch 4/100, Train Loss: 1.0268, Test MSE: 0.6983, Test CI: 0.7613
Epoch 5/100, Train Loss: 1.0011, Test MSE: 0.6050, Test CI: 0.7670
Epoch 6/100, Train Loss: 0.9823, Test MSE: 0.6106, Test CI: 0.7732
Epoch 7/100, Train Loss: 0.9913, Test MSE: 0.5937, Test CI: 0.7777
Epoch 8/100, Train Loss: 0.9818, Test MSE: 0.5914, Test CI: 0.7802
Epoch 9/100, Train Loss: 0.9552, Test MSE: 0.5918, Test CI: 0.7828
Epoch 10/100, Train Loss: 0.9618, Test MSE: 0.5818, Test CI: 0.7821
Epoch 11/100, Train Loss: 0.9467, Test MSE: 0.5954, Test CI: 0.7806
Epoch 12/100, Train Loss: 0.9537, Test MSE: 0.6222, Test CI: 0.7836
Epoch 13/100, Train Loss: 0.9395, Test MSE: 0.6300, Test CI: 0.7818
Epoch 14/100, Train Loss: 0.9346, Test MSE: 0.5784, Test CI: 0.7873
Epoch 15/100, Train Loss: 0.9202, Test MSE: 0.5649, Test 

In [ ]:
import torch.nn.functional as F

class LSTMWithCrossAttention(nn.Module):
    def __init__(self, config):
        super(LSTMWithCrossAttention, self).__init__()
        self.drug_embedding = nn.Embedding(config['charsmiset_size'] + 1, config['embed_dim'])
        self.protein_embedding = nn.Embedding(config['charseqset_size'] + 1, config['embed_dim'])

        self.drug_lstm = nn.LSTM(config['embed_dim'], config['lstm_dim'],
                                 num_layers=config['lstm_layers'], batch_first=True, bidirectional=True)
        self.protein_lstm = nn.LSTM(config['embed_dim'], config['lstm_dim'],
                                    num_layers=config['lstm_layers'], batch_first=True, bidirectional=True)

        self.drug_attention = nn.Linear(config['lstm_dim'] * 2, config['lstm_dim'] * 2)
        self.protein_attention = nn.Linear(config['lstm_dim'] * 2, config['lstm_dim'] * 2)

        self.fc1 = nn.Linear(config['lstm_dim'] * 4, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

        self.dropout = nn.Dropout(config['dropout_rate'])

    def cross_attention(self, query, key, value):
        # query: [batch_size, query_len, dim]
        # key: [batch_size, key_len, dim]
        # value: [batch_size, key_len, dim]

        attention_scores = torch.bmm(query, key.transpose(1, 2))
        attention_weights = F.softmax(attention_scores, dim=-1)
        context_vector = torch.bmm(attention_weights, value)
        return context_vector

    def forward(self, drug, protein):
        drug_embedded = self.drug_embedding(drug)
        protein_embedded = self.protein_embedding(protein)

        drug_output, _ = self.drug_lstm(drug_embedded)
        protein_output, _ = self.protein_lstm(protein_embedded)

        # Cross-attention
        drug_query = self.drug_attention(drug_output)
        protein_query = self.protein_attention(protein_output)

        drug_context = self.cross_attention(drug_query, protein_output, protein_output)
        protein_context = self.cross_attention(protein_query, drug_output, drug_output)

        # Global average pooling
        drug_repr = torch.mean(drug_context, dim=1)
        protein_repr = torch.mean(protein_context, dim=1)

        concat = torch.cat([drug_repr, protein_repr], dim=1)

        fc1 = F.relu(self.fc1(concat))
        fc1 = self.dropout(fc1)
        fc2 = F.relu(self.fc2(fc1))
        fc2 = self.dropout(fc2)
        output = self.fc3(fc2)

        return output.squeeze(-1)

config = {
        'charsmiset_size': 64,
        'charseqset_size': 26,
        'embed_dim': 128,
        'lstm_dim': 64,
        'lstm_layers': 2,
        'dropout_rate': 0.3,
        'learning_rate': 0.0005,
        'batch_size': 128,

        # Dataset specific
        'davis_convert_to_log': True,
        'kiba_convert_to_log': False,

        # Input dimensions
        'max_seq_len': 1000,
        'max_smi_len': 100,
        'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
        'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
        'problem_type': 1

}

train_dataset, test_dataset = get_datasets('davis', config)


mse, ci = run_experiment(LSTMWithCrossAttention, config, train_dataset, test_dataset)
print(f"LSTM with Cross-Attention - Final MSE: {mse:.4f}, Final CI: {ci:.4f}")

Epoch 1/100, Train Loss: 3.7172, Test MSE: 0.7958, Test CI: 0.5358
Epoch 2/100, Train Loss: 1.3072, Test MSE: 0.7662, Test CI: 0.6014
Epoch 3/100, Train Loss: 1.2361, Test MSE: 0.7402, Test CI: 0.6447
Epoch 4/100, Train Loss: 1.1697, Test MSE: 0.7410, Test CI: 0.6977
Epoch 5/100, Train Loss: 1.1333, Test MSE: 0.6272, Test CI: 0.7312
Epoch 6/100, Train Loss: 1.0788, Test MSE: 0.5924, Test CI: 0.7552
Epoch 7/100, Train Loss: 1.0467, Test MSE: 0.6160, Test CI: 0.7737
Epoch 8/100, Train Loss: 1.0219, Test MSE: 0.5764, Test CI: 0.7764
Epoch 9/100, Train Loss: 1.0198, Test MSE: 0.5644, Test CI: 0.7882
Epoch 10/100, Train Loss: 0.9859, Test MSE: 0.5471, Test CI: 0.7868
Epoch 11/100, Train Loss: 0.9643, Test MSE: 0.5394, Test CI: 0.7866
Epoch 12/100, Train Loss: 0.9513, Test MSE: 0.5298, Test CI: 0.7910
Epoch 13/100, Train Loss: 0.9332, Test MSE: 0.5496, Test CI: 0.7964
Epoch 14/100, Train Loss: 0.9268, Test MSE: 0.5153, Test CI: 0.8012
Epoch 15/100, Train Loss: 0.9037, Test MSE: 0.5282, Test 

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerWithCrossAttention(nn.Module):
    def __init__(self, config):
        super(TransformerWithCrossAttention, self).__init__()
        self.drug_embedding = nn.Embedding(config['charsmiset_size'] + 1, config['d_model'])
        self.protein_embedding = nn.Embedding(config['charseqset_size'] + 1, config['d_model'])

        self.pos_encoder = PositionalEncoding(config['d_model'], config['dropout_rate'])

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config['d_model'],
            nhead=config['n_head'],
            dim_feedforward=config['d_ff'],
            dropout=config['dropout_rate'],
            batch_first=True
        )

        self.drug_encoder = nn.TransformerEncoder(encoder_layer, num_layers=config['n_layers'])
        self.protein_encoder = nn.TransformerEncoder(encoder_layer, num_layers=config['n_layers'])

        self.cross_attention = nn.MultiheadAttention(
            embed_dim=config['d_model'],
            num_heads=config['n_head'],
            dropout=config['dropout_rate'],
            batch_first=True
        )

        self.fc1 = nn.Linear(config['d_model'] * 2, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

        self.dropout = nn.Dropout(config['dropout_rate'])

    def forward(self, drug, protein):
        drug_embedded = self.drug_embedding(drug)
        protein_embedded = self.protein_embedding(protein)

        # Apply positional encoding
        drug_embedded = self.pos_encoder(drug_embedded.transpose(0, 1)).transpose(0, 1)
        protein_embedded = self.pos_encoder(protein_embedded.transpose(0, 1)).transpose(0, 1)

        drug_output = self.drug_encoder(drug_embedded)
        protein_output = self.protein_encoder(protein_embedded)

        # Cross-attention using MultiheadAttention
        drug_context, _ = self.cross_attention(drug_output, protein_output, protein_output)
        protein_context, _ = self.cross_attention(protein_output, drug_output, drug_output)

        # Global average pooling
        drug_repr = torch.mean(drug_context, dim=1)
        protein_repr = torch.mean(protein_context, dim=1)

        concat = torch.cat([drug_repr, protein_repr], dim=1)

        fc1 = F.relu(self.fc1(concat))
        fc1 = self.dropout(fc1)
        fc2 = F.relu(self.fc2(fc1))
        fc2 = self.dropout(fc2)
        output = self.fc3(fc2)

        return output.squeeze(-1)

config = {
    'charsmiset_size': 64,
    'charseqset_size': 26,
    'd_model': 128,  # Dimension of the model
    'n_head': 8,     # Number of attention heads
    'n_layers': 2,   # Number of Transformer layers
    'd_ff': 512,     # Dimension of the feedforward network in Transformer
    'dropout_rate': 0.1,
    'learning_rate': 0.0001,
    'batch_size': 64,

    # Dataset specific
    'davis_convert_to_log': True,
    'kiba_convert_to_log': False,

    # Input dimensions
    'max_seq_len': 1000,
    'max_smi_len': 100,
    'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
    'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
    'problem_type': 1

}

train_dataset, test_dataset = get_datasets('davis', config)

mse, ci = run_experiment(TransformerWithCrossAttention, config, train_dataset, test_dataset)
print(f"Transformer with Cross-Attention - Final MSE: {mse:.4f}, Final CI: {ci:.4f}")

Epoch 1/100, Train Loss: 3.0500, Test MSE: 0.8052, Test CI: 0.5697
Epoch 2/100, Train Loss: 0.9485, Test MSE: 0.7775, Test CI: 0.6392
Epoch 3/100, Train Loss: 0.8436, Test MSE: 0.6742, Test CI: 0.7166
Epoch 4/100, Train Loss: 0.7923, Test MSE: 0.6392, Test CI: 0.7362
Epoch 5/100, Train Loss: 0.7577, Test MSE: 0.6726, Test CI: 0.7464
Epoch 6/100, Train Loss: 0.7575, Test MSE: 0.6038, Test CI: 0.7516
Epoch 7/100, Train Loss: 0.7314, Test MSE: 0.5865, Test CI: 0.7575
Epoch 8/100, Train Loss: 0.6987, Test MSE: 0.5846, Test CI: 0.7689
Epoch 9/100, Train Loss: 0.6956, Test MSE: 0.5842, Test CI: 0.7769
Epoch 10/100, Train Loss: 0.6743, Test MSE: 0.6608, Test CI: 0.7737
Epoch 11/100, Train Loss: 0.6691, Test MSE: 0.5714, Test CI: 0.7823
Epoch 12/100, Train Loss: 0.6574, Test MSE: 0.5525, Test CI: 0.7869
Epoch 13/100, Train Loss: 0.6528, Test MSE: 0.5301, Test CI: 0.7951
Epoch 14/100, Train Loss: 0.6437, Test MSE: 0.5582, Test CI: 0.7846
Epoch 15/100, Train Loss: 0.6435, Test MSE: 0.5317, Test 

In [ ]:
config = {
    'charsmiset_size': 64,
    'charseqset_size': 26,
    'd_model': 128,  # Dimension of the model
    'n_head': 8,     # Number of attention heads
    'n_layers': 2,   # Number of Transformer layers
    'd_ff': 512,     # Dimension of the feedforward network in Transformer
    'dropout_rate': 0.3,
    'learning_rate': 0.0005,
    'batch_size': 128,

    # Dataset specific
    'davis_convert_to_log': True,
    'kiba_convert_to_log': False,

    # Input dimensions
    'max_seq_len': 1000,
    'max_smi_len': 100,
    'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
    'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
    'problem_type': 1

}

train_dataset, test_dataset = get_datasets('davis', config)


mse, ci = run_experiment(TransformerWithCrossAttention, config, train_dataset, test_dataset)
print(f"Transformer with Cross-Attention - Final MSE: {mse:.4f}, Final CI: {ci:.4f}")

Epochs:   1%|          | 1/100 [00:46<1:16:06, 46.13s/it]

Epoch 1/100, Train Loss: 2.6403, Test MSE: 0.9272, Test CI: 0.5660


Epochs:   2%|▏         | 2/100 [01:32<1:15:16, 46.09s/it]

Epoch 2/100, Train Loss: 1.3347, Test MSE: 1.2470, Test CI: 0.5739


Epochs:   3%|▎         | 3/100 [02:18<1:14:28, 46.07s/it]

...

Epoch 98/100, Train Loss: 0.4030, Test MSE: 0.4630, Test CI: 0.8289


Epochs:  99%|█████████▉| 99/100 [1:16:02<00:46, 46.05s/it]

Epoch 99/100, Train Loss: 0.3956, Test MSE: 0.5091, Test CI: 0.8112


Epochs: 100%|██████████| 100/100 [1:16:49<00:00, 46.09s/it]

Epoch 100/100, Train Loss: 0.3867, Test MSE: 0.4676, Test CI: 0.8262
Best Test MSE: 0.4233, Best Test CI: 0.8157
Transformer with Cross-Attention - Final MSE: 0.4233, Final CI: 0.8157


In [ ]:
# Tested Different Architectures, Bi-Directional LSTM With Cross Attention Was Best

In [ ]:
config = {
        # Model architecture
        'model_type': 'lstm',
        'lstm_layers': 1,
        'lstm_dim': 32,
        'dense_layers': 1,
        'dense_dim': 32,
        'dropout_rate': 0.2,
        'embed_dim': 64,

        # Training parameters
        'learning_rate': 0.001,
        'batch_size': 64,
        'num_epochs': 100,

        # Input dimensions
        'max_seq_len': 1000,
        'max_smi_len': 100,

        # Data handling
        'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
        'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
        'problem_type': 1,
        'binary_th': 0.0,

        # Output and logging
        'checkpoint_path': '',
        'log_dir': 'logs',

        # Dataset specific
        'davis_convert_to_log': True,
        'kiba_convert_to_log': False,

        # Alphabet sizes
        'charsmiset_size': 64,  # Size of SMILES alphabet
        'charseqset_size': 26,  # Size of protein sequence alphabet
    }

dataset = DataSet(config['kiba_path'], config['problem_type'], config['max_seq_len'],
                      config['max_smi_len'], 'kiba', config['kiba_convert_to_log'])

XD, XT, Y, label_row_inds, label_col_inds, test_fold, train_folds = dataset.get_data()

train_fold = unite_lists(train_folds)

train_drugs, train_targets, train_affinity = prepare_interaction_pairs(XD, XT, Y, label_row_inds[train_fold], label_col_inds[train_fold])
test_drugs, test_targets, test_affinity = prepare_interaction_pairs(XD, XT, Y, label_row_inds[test_fold], label_col_inds[test_fold])

train_dataset_kiba = DTIDataset(train_drugs, train_targets, train_affinity)
test_dataset_kiba = DTIDataset(test_drugs, test_targets, test_affinity)

Reading kiba dataset from /content/drive/MyDrive/BioInf-Final/data/kiba/
Parsing kiba dataset


In [ ]:
config = {
        'charsmiset_size': 64,
        'charseqset_size': 26,
        'embed_dim': 128,
        'lstm_dim': 64,
        'lstm_layers': 2,
        'dropout_rate': 0.3,
        'learning_rate': 0.0005,
        'batch_size': 128,

        # Dataset specific
        'davis_convert_to_log': True,
        'kiba_convert_to_log': False,

        # Input dimensions
        'max_seq_len': 1000,
        'max_smi_len': 100,
        'davis_path': '/content/drive/MyDrive/BioInf-Final/data/davis/',
        'kiba_path': '/content/drive/MyDrive/BioInf-Final/data/kiba/',
        'problem_type': 1

}

train_dataset, test_dataset = get_datasets('kiba', config)


mse, ci = run_experiment(LSTMWithCrossAttention, config, train_dataset, test_dataset)
print(f"LSTM with Cross-Attention - Final MSE: {mse:.4f}, Final CI: {ci:.4f}")

Epochs:   1%|          | 1/100 [05:15<8:41:12, 315.88s/it]

Epoch 1/100, Train Loss: 6.3522, Test MSE: 0.6836, Test CI: 0.5935


Epochs:   2%|▏         | 2/100 [10:31<8:35:29, 315.61s/it]

Epoch 2/100, Train Loss: 2.8758, Test MSE: 0.6793, Test CI: 0.6408


Exception ignored in: <function _xla_gc_callback at 0x7854a480d510>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Epochs:   3%|▎         | 3/100 [15:45<8:28:49, 314.74s/it]

Epoch 3/100, Train Loss: 2.7781, Test MSE: 0.6370, Test CI: 0.6548


Epochs:   4%|▍         | 4/100 [20:57<8:22:17, 313.93s/it]

Epoch 4/100, Train Loss: 2.6347, Test MSE: 0.7229, Test CI: 0.6784


In [ ]:
# Epoch 1/100, Train Loss: 2.9745, Test MSE: 0.7973, Test CI: 0.5264
# Epoch 2/100, Train Loss: 1.3261, Test MSE: 0.7863, Test CI: 0.5359